<img src="../resources/cropped-SummerWorkshop_Header.png">  

<h1 align="center">Brain Observatory - Visual Behavior </h1> 
<h2 align="center">Summer Workshop on the Dynamic Brain </h2> 
<h3 align="center">Monday, August 26, 2019</h3> 

<img src="../resources/visual_behavior_experiment.png" height="400" width="1200">  


This notebook will introduce you to the Visual Behavior Brain Observatory dataset. This dataset uses 2-photon calcium imaging (also called optical physiology or ophys) to measure neural activity in mice performing a visual change detection task. One aim of this dataset is to ask: how is sensory coding influenced by expectation, engagement, and experience?

The change detection task consists of a series of image presentations. Each image flash is 250ms followed by 500ms of gray screen. The task for the mouse is to lick in a 750ms response window following a change in image identity. On each trial, a change time is scheduled. On go trials, a change in image identity occurs. On catch trials, no image change occurs (aka 'sham change'), and we measure false alarm rates in the same 750ms response window. Correct responses are rewarded and licks outside the response window result in a timeout.

There are 8 natural scene images shown in each behavioral session. Mice learn the task with one set of 8 natural scenes which become highly familiar with experience. During the imaging phase of the experiment, mice perform the task with the familiar image set, as well as another set of 8 images that are experienced for the first time under the microscope. This allows us to ask how training history and visual experience infuence sensory responses. 

There are 2 types of sessions during the imaging portion of the experiment - active behavior and passive viewing. During the passive viewing sessions, the task is run in open loop mode with the lick spout retracted, after the mouse has been give its daily allocation of water. This allows us to ask how representations differ when the mouse is actively engaged in the task and motivated to earn water rewards compared to when it is sated and not receiving reward feedback.

During imaging sessions, 5% of non-change image flashes are randomly omitted from the otherwise regular sequence of stimulus presentations. This allows us to ask whether expectation signals are present in the visual cortex. 

The dataset consists of recordings from excitatory (Slc17a7-IRES2-Cre;CaMK2-tTA;Ai93(GCaMP6f)) and VIP inhibitory (VIP-IRES-Cre;Ai162(GCaMP6f)) neurons in V1. Excitatory cells were sampled at 2 depths: 175um (L2/3) and 375um (L5). VIP cells were sampled at 175um depth.

In this notebook, we will describe the core components of each experimental session and the tools for accessing and analyzing the data.

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">

<p>Let's get started

</div>

In [ ]:
# you will need these libraries for computation & data manipulation
import os
import numpy as np
import pandas as pd

# matplotlib is a standard python visualization package
import matplotlib.pyplot as plt
%matplotlib inline 

# seaborn is another library for statistical data visualization
# seaborn style & context settings make plots pretty & legible
import seaborn as sns
sns.set_context('notebook', font_scale=1.5, rc={'lines.markeredgewidth': 2})
sns.set_style('white')
sns.set_palette('deep');

In [ ]:
# Import allensdk modules for loading and interacting with the data
from allensdk.brain_observatory.behavior.swdb import behavior_project_cache as bpc

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<p>The first thing we will do is use the <code>allensdk</code> to load a cache for the visual behavior dataset, which contains a dataframe describing the dimensions of the dataset and methods for loading the data from particular sessions. You can inspect the <code>experiment_table</code> contained in the cache to identify experiments of interest and their metadata. 

</div>

In [ ]:
# AWS path
cache_path = r'/data/dynamic-brain-workshop/visual_behavior/2019'

# # Mac/Linux path
# cache_path = r'/Volumes/Brain2019/dynamic-brain-workshop/visual_behavior/2019'

# # Windows path
# cache_path = r'H:\dynamic-brain-workshop\visual_behavior\2019'

cache = bpc.BehaviorProjectCache(cache_path)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.1:  Get information about what's in the dataset </b>

<p>Read in <code>experiment_table</code> using the cache object and explore the columns to see the available visual areas, cre lines, and session types. 

</div>

In [ ]:
# get the table of all experiment sessions for this dataset
experiments = cache.experiment_table
experiments.head(10)

In [ ]:
experiments = cache.experiment_table
experiments.columns

In [ ]:
# what are the dimensions of this dataset? 
print('targeted structures:', experiments.targeted_structure.unique())
print('\ncre_lines:', experiments.full_genotype.unique())
print('\nstage_types:', experiments.stage_name.unique())

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.2: Get your experiment </b>

<p>Get a random experiment ID for an active behavior session and assign it to a variable called <code>experiment_id</code>. Use pandas <code>sample()</code> to get a random experiment from a given column.
  
 __[Documentation for pandas.sample() ](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.sample.html)__ 


<p>What is the <code>targeted_structure</code>, <code>imaging_depth</code>, <code>full_genotype</code>, and <code>stage_name</code> for your <code>experiment_id</code>? 
    
</div>

In [ ]:
# get a random active behavior experiment
active_experiments = experiments[experiments.passive_session==False]
experiment_id = active_experiments.ophys_experiment_id.sample(1).values[0]

In [ ]:
# get the metadata for this experiment from the manifest


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 1.3:  What is in an experiment container? </b>

<p>The experiment container describes a set of imaging sessions performed at the same location (targeted structure and imaging depth) in the same mouse that targets the same set of cells. All the sessions in an experiment container have a common <code>experiment_container_id</code>.

<p>Get a the <code>experiment_container_id</code> for your <code>experiment_id</code> and find out what other sessions were recorded at that same location. 

<p>Do all experiment containers have the same number of sessions associated with them? 
    
</div>

In [ ]:
# get the container ID for this experiment
container_id = experiments[experiments.ophys_experiment_id==experiment_id]['container_id'].values[0]

In [ ]:
# what other experiment sessions are in this container?
experiments.groupby('container_id').get_group(container_id)

In [ ]:
# Get number of sessions in each container
experiments.groupby('container_id').size()

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Behavior OPhys Session object</h2>
<p>The BehaviorOphysSession class in allensdk.brain_observatory.behavior.behavior_ophys_session provides an interface to all of the data for a single experimental session from the Visual Behavior pipeline, aligned to a common time clock.

<p>We package each session's data into a Neurodata Without Borders 2.0 (NWB) file. The BehaviorOphysSession will load data from the NWB file for a given session.
    
<p>You can load a BehaviorOphysSession object easily using the 'get_session' method of the cache object. 

<p>Use help() to view documentation on the session object. 

</div>

In [ ]:
# get a session from the cache
session = cache.get_session(experiment_id)

In [ ]:
help(session)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.1:  What is an experiment session? </b>

<p>Use tab completion to see what is in the dataset object for an experiment session

<p>What is in the <code>metadata</code> attribute? What is in the <code>task_parameters</code> attribute?

</div>


In [ ]:
# get session metadata


In [ ]:
# get session task parameters


<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Optical physiology data - max projection, roi masks, and fluorescence traces</h2>

<p>Let's use the session object to access neuron fluorescence timeseries, roi masks, and metadata. An ROI mask is used to define the boundary of each cell in the flourescence data. The timeseries extracted from each ROI is one cell's activity.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.2: Max intensity projection and ROI masks</b>
    
<p>Get the maximum intensity projection image using the <code>max_projection</code> attribute for your dataset and display it. 
    
<p>Get the <code>segmentation_mask_image</code> and display it next to the max projection.

</div>

In [ ]:
# plot the max intensity projection and the segmentation mask
fig, ax = plt.subplots(1, 2)
fig.set_size_inches(10, 5)
ax[0].imshow(session.max_projection)
ax[1].imshow(session.segmentation_mask_image)
plt.tight_layout()

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">    
<p>Get ROI mask for a single cell using the <code>roi_masks</code> attribute. 
</div>

In [ ]:
cell_specimen_ids = list(session.roi_masks.keys())
plt.imshow(session.roi_masks[cell_specimen_ids[9]]);

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.3: Get dF/F traces and ophys timestamps</b>

<p>Get <code>dff_traces</code> and <code>ophys_timestamps</code> attributes. How are they formatted?

<p><code>dff_traces</code> is a dataframe with <code>cell_specimen_id</code> as the index and a column called <code>dff</code> which contains the baseline normalized fluorescence traces, also called dF/F traces, for each cell in the session. 
    
<p><code>ophys_timestamps</code> is an array of timestamps corresponding to each 2P imaging frame. 
    
<p>Check that the length of one of the dF/F traces is the same length as the ophys timestamps.

</div>

In [ ]:
# get dff_traces 


In [ ]:
# get the ophys_timestamps

In [ ]:
# get shape of traces and timestamps
print('shape of dff_traces:',session.dff_traces.shape)
print('shape of ophys_timestamps:',session.ophys_timestamps.shape)

In [ ]:
# shape of one cell's trace
print('shape of one trace:',session.dff_traces.iloc[0]['dff'].shape)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 2.4: Plot the dF/F trace for a cell</b>

<p>Plot the dF/F trace for one cell by indexing into the <code>dff_traces</code> array. You can get a specific row of the dataframe with .iloc, or use the cell_specimen_id to index with the .loc method. 
    
 __[Difference between .iloc and .loc](https://stackoverflow.com/questions/31593201/how-are-iloc-ix-and-loc-different)__ 
    
<p>Use <code>ophys_timestamps</code> to plot the x-axis in seconds. 
    
<p>Try plotting the trace for a few different cells.

</div>

In [ ]:
# plot the dF/F trace for one cell using ophys timestamps for x-axis values
# indexing method using row index with .iloc
cell_index = 0 
dff_trace = session.dff_traces.iloc[cell_index]['dff'] # note that the column name is outside of the .iloc call
plt.plot(session.ophys_timestamps, dff_trace)
plt.xlabel('time (sec)');
plt.ylabel('dF/F');
plt.title('cell index: '+str(cell_index));

In [ ]:
# plot the dF/F trace for one cell using ophys timestamps for x-axis values
# indexing method using cell_specimen_id as index with .loc

# get cell_specimen_id from a list of all cell_specimen_ids
cell_specimen_ids = session.dff_traces.index.values
cell_specimen_id = cell_specimen_ids[cell_index]

dff_trace = session.dff_traces.loc[cell_specimen_id, 'dff'] #note how the column name is included in the .loc call
plt.plot(session.ophys_timestamps, dff_trace)
plt.xlabel('time (sec)');
plt.ylabel('dF/F');
plt.title('cell_specimen_id: '+str(cell_specimen_id));

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b>Task 2.5: Plot a heatmap of all cell traces in this session</b>

<p>Extract the <code>dff_traces</code> from the dataframe into an array using <code>np.vstack()</code>. What is the shape?

<p>Use the matplotlib plotting function <code>pcolormesh</code> to plot the matrix as a heatmap. Plot the x-axis in seconds.

</div>

In [ ]:
# turn dff_traces into an array of cells x timepoints
dff_traces_array = np.vstack(session.dff_traces.dff.values)
print('shape of dff_traces_array:',dff_traces_array.shape)

In [ ]:
# plot a heatmap of all traces 
fig, ax = plt.subplots(figsize=(20,5))
cax = ax.pcolormesh(dff_traces_array, cmap='magma', vmin=0, vmax=np.percentile(dff_traces_array, 99))
ax.set_yticks(np.arange(0, len(dff_traces_array)), 10);
ax.set_ylabel('cells')
ax.set_xlabel('time (sec)')
ax.set_xticks(np.arange(0, len(session.ophys_timestamps), 600*31));
ax.set_xticklabels(np.arange(0, session.ophys_timestamps[-1], 600));
cb = plt.colorbar(cax, pad=0.015, label='dF/F')

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Behavior timeseries and events - running, licks, and rewards </h2>
<p>As the mouse performs the behavioral task, it is free to run on a disk. The task is a go/no-go style task with licking as the behavioral response. When a mouse correctly licks the water spout, a reward is delivered. 

<p>Running, licks and rewards are measured at the stimulus frame display rate and share timestamps with the stimulus. </div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.1: Get running speed trace and timestamps</b>

<p>Get the <code>running_speed</code> attribute of the dataset object. What does it contain? 

<p>Runnning speed shares timestamps with the visual stimulus. Compare the values of running timestamps from  <code>running_speed</code> with the values in the dataset attribute <code>stimulus_timestamps</code>. Note that all timestamps are in seconds. 
    
</div>

In [ ]:
# get running speed


In [ ]:
# what are the values of running speed timestamps?
print('running speed timestamps:',session.running_speed.timestamps.values)

In [ ]:
# what are the values of stimulus timestamps?
print('stimulus timestamps:',session.stimulus_timestamps)

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.2: Plot running speed</b>

<p>Plot the values for <code>running_speed</code> with time in seconds on the x-axis. 
    
<p>Running speed is measured in cm/s. Label the axes appropriately.
        
</div>

In [ ]:
# plot running speed with timestamps on x-axis


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 3.3: Rewards and licks</b>
    
<p>Get the <code>rewards</code> attribute of the session object. How is it formatted? 

<p>Get the <code>licks</code> attribute of the session object. How is it formatted? 
    
</div>

In [ ]:
# Get information about rewards


In [ ]:
# Get information about licks


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b>Task 3.4: Plot licking, reward times, and running speed</b>
    
<p>1) Plot <code>running_speed</code> as above, but set xlims to focus on a 30 second portion of the behavior session (ex: from x=600 to x=630). 

<p>2) On the same plot, plot <code>rewards</code> as points (not a line), at y = -10. Note that <code>rewards</code> is a dataframe, with timestamps as the index. Use the values of the index to get the times of all rewards to plot along the x-axis.

<p>Hint: You will need to create an array of len(session.rewards.index.values) filled with -10 to use as y-axis values to plot. np.repeat() is a convenient function for this.

<p>3) Add times of <code>licks</code> using plt.vlines() to your plot, with ymin=-10 and ymax=-5. 

<p>What is the relationship between running, licking and rewards? 
    
</div>

In [ ]:
# plot running speed, rewards, and licks
plt.plot(session.running_speed.timestamps, session.running_speed.speed)
plt.ylabel('running speed (cm/s)')
plt.xlabel('time (sec)')
plt.xlim(600,630)
plt.plot(session.rewards.index.values, np.repeat(-10, np.shape(session.rewards.index.values)), 'o')
plt.vlines(session.licks,ymin=-10, ymax=-5)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Visual stimuli </h2>
    
<p>The timing of visual stimui can be accessed through the 'stimulus_presentations' table. This includes the timing of omitted stimuli - in other words, the time where the image would have been presented if it were not omitted.  
    
<p>The images shown during the session are included in the 'stimulus_template'. 

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 4.1: Get the stimulus table</b>

<p>Get the <code>stimulus_presentations</code> attribute to identify the times of stimulus presentations. How many stimulus flashes were there? 

<p>What other data is included for each stimulus flash in this table? What could it be used for?
    
</div>

In [ ]:
# get the stimulus presentations table


In [ ]:
# how many stimulus presentations were there? 


In [ ]:
# what are the columns of the stimulus presentations table?


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 4.2: Plot visual stimulus presentations with behavior events</b>

<p>1) Copy and paste your code from Task 3.5

<p>2) On the same plot, plot stimulus presentations using the <code>start_time</code> and <code>stop_time</code> columns with plt.axvspan(). Set alpha=0.3 & facecolor='gray'.

<p>Hint: Loop through each row of the stimulus table using the pandas method <code>iterrows</code> to plot all stimulus flashes. 
    
 __[Documentation on pandas.iterrows()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.iterrows.html)__ 
    
<p>3) Bonus: Plot stimulus presentations corresponding to image changes using the <code>change</code> column. Set facecolor='blue' to distinguish from non-change flashes. 

</div>

In [ ]:
# plot running, rewards, licks and stimuli using axvspan to delineate periods where a stimulus was shown
plt.plot(session.running_speed.timestamps, session.running_speed.speed)
plt.ylabel('running speed (cm/s)')
plt.xlabel('time (sec)')
plt.xlim(600,630)
plt.plot(session.rewards.index.values, np.repeat(-10, np.shape(session.rewards.index.values)), 'o')
plt.vlines(session.licks,ymin=-10, ymax=-5)

for index, row in session.stimulus_presentations.iterrows():
    plt.axvspan(row.start_time, row.stop_time,alpha=0.3,facecolor='gray')
    if row.change:
        plt.axvspan(row.start_time, row.stop_time,alpha=0.3,facecolor='blue')

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 4.3: Get visual stimulus templates</b>

<p>Get the <code>stimulus_templates</code> from the session object. How is it formatted? 

<p>The keys of the <code>stimulus_templates</code> dictionary correspond to the <code>image_names</code> in <code>stimulus_presentations</code>. The values are the images as arrays. 
    
<p>Plot an image from <code>stimulus_templates</code>. Show the name of the image in the title.
    
</div>

In [ ]:
# get the stimulus templates


In [ ]:
# plot a stimulus using its image_name
stimuli = session.stimulus_presentations.copy()
image_name = stimuli[stimuli.image_name!='omitted'].image_name.unique()[0]
plt.imshow(session.stimulus_templates[image_name], cmap='gray')
plt.title(image_name);

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>Behavior trials data</h2>
    
<p>The <code>trials</code> dataframe organizes behavior events (including licking and rewards), stimulus information (what stimulus was shown before and after the scheduled change time) and metadata (such as whether the trial was a 'go' trial or a 'catch' trial) for each behavioral trial. 

<p>This structure is convenient for data exploration and analysis.
</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 5.1: Explore the trials table</b>

<p>1) Get the <code>trials</code> attribute of the <code>session</code> object. What are the columns of this dataframe? What are the rows?

<p>2) How many go trials were there? How many catch trials? What is the ratio of go to catch trials?

<p>3) What images were shown in this behavior session? Use the pandas <code>unique()</code> method to get the unique images from the trials table. 
</div>

In [ ]:
# get the trials table 


In [ ]:
# how many go trials were there? 


In [ ]:
# how many catch trials were there?


In [ ]:
# what images were shown? 


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 5.2: Get the hit and false alarm rates for this session</b>

<p>The hit rate is the fraction of go trials with a lick in the reward window
    
<p>The false alarm rate is the fraction of catch trials with a lick in the reward window

<p>1) Select all the go trials by filtering the dataframe by <code>go</code> = True. Get the fraction of go trials where <code>hit</code> = True. 

<p>2) Repeat for catch trials.

</div>

In [ ]:
# compute the hit rate for go trials
go_trials = session.trials[session.trials.go]
print(len(go_trials), 'go trials')
print(np.sum(go_trials.hit), 'hits')
print('hit rate:', round(np.sum(go_trials.hit)/len(go_trials),2))

In [ ]:
# compute the false alarm rate for catch trials
catch_trials = session.trials[session.trials.catch]
print(len(catch_trials),'catch trials')
print(np.sum(catch_trials.false_alarm),'false alarms')
print('false alarm rate:',round(np.sum(catch_trials.false_alarm)/len(catch_trials),2))

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 5.3: Plot a lick raster across trials aligned to the change time</b>

<p>Provide the <code>trials</code> dataframe to the function below to plot a lick raster.

<p>Is the mouse performing the task consistently across the whole session?
</div>

In [ ]:
def make_lick_raster(trials):
    trials = trials[trials.aborted==False]
    trials = trials.reset_index()
    fig,ax = plt.subplots(figsize=(5,10))
    for trial_index, trial_data in trials.iterrows(): 
        # get times relative to change time
        lick_times = [(t - trial_data.change_time) for t in trial_data.lick_times]
        reward_time = [(t - trial_data.change_time) for t in [trial_data.reward_time]]
        # plot reward times
        if len(reward_time) > 0:
            ax.plot(reward_time[0], trial_index + 0.5, '.', color='b', label='reward', markersize=6)
        # plot lick times
        ax.vlines(lick_times, trial_index, trial_index + 1, color='k', linewidth=1)
        # put a line at the change time
        ax.vlines(0, trial_index, trial_index + 1, color=[.5, .5, .5], linewidth=1)
    # gray bar for response window
    ax.axvspan(0.1, 0.7, facecolor='gray', alpha=.3, edgecolor='none')
    ax.grid(False)
    ax.set_ylim(0, len(trials))
    ax.set_xlim([-1, 4])
    ax.set_ylabel('trials')
    ax.set_xlabel('time relative to change (sec)')
    ax.set_title('lick raster')
    plt.gca().invert_yaxis()

In [ ]:
# plot the lick raster for this session using the provided function
make_lick_raster(session.trials)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF; ">
<h2>The Trial Response and Flash Response dataframes organize cell responses by behavior trials and stimulus flashes </h2>
    
<p> We have done the work of temporal alignment for you to create these two dataframes that provide a convenient data structure for analysis. 
  
<p> The <code>trial_response_df</code> extracts cell responses for each behavioral trial in a [-4,8] second window around the change time.
    
<p> The <code>flash_response_df</code> extracts cell responses for each stimulus presentation in a [-0.5, 0.75] second window around each flash. 
    
<p> Both dataframes take the mean response for each cell in a 500ms window after the change time for trials, or after the stimulus onset time for stimulus presentations.
    
<p> These dataframes also include a column called <code>p_value</code>, comparing the response for each cell on each trial to a shuffled distribution from the spontaneous activity epochs. There is also a column called <code>pref_stim</code> with a Boolean indicating the image that drove the strongest mean response for each cell.  

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 6.1: Load and explore the Trial Response Dataframe</b> 

<p>1) Get the <code>trial_response_df</code> attribute of the session object. What are the columns? What are the rows? What is different than the <code>trials</code> table? 
    
<p>The <code>dff_trace</code> column contains a portion of each cell's dF/F trace from 4 seconds before the <code>change_time</code> to 8 seconds after the <code>change_time</code> for each trial. There are also <code>dff_trace_timestamps</code> for the same window. 

<p> For each trial, the <code>mean_response</code> of each cell is computed for a 500ms window after the <code>change_time</code>.

<p>2) Assign <code>trial_response_df</code> to a variable named <code>tr</code> for convenient use in later exercises.
    
</div>

In [ ]:
# get the trial response dataframe and assign it to 'tr'


In [ ]:
# what is in the trial response dataframe?


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 6.2: Plot the population average trace for go trials </b>

<p>Select go trials from the <code>trial_response_df</code>, take the mean across all cells, all trials, and plot it. 
    
<p>Bonus: plot the x-axis in seconds relative to the change time
    
</div>

In [ ]:
# plot the mean trace across all cells for go trials
fig, ax = plt.subplots()

traces = tr[tr.go==True].dff_trace.values
mean_trace = np.mean(traces, axis=0)
time_seconds = tr.iloc[0].dff_trace_timestamps  - tr.iloc[0].change_time
ax.plot(time_seconds, mean_trace)
ax.set_xlabel('time (sec)');
ax.set_ylabel('mean dF/F');

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
<p><b>Task 6.3: Plot the population response across cells for one change trial</b>

<p>1) Select one <code>trial_id</code> where <code>go</code> = True and filter the <code>trial_response_df</code> to get the data for just that trial. How many rows are in this subset of data? Is it the same length as the number of unique cells?

<p>2) Get the <code>mean_response</code> for all cells,  sort in order of response magnitude and plot it. 
    
</div>

In [ ]:
# get data for all cells for a single go trial
go_trials = tr[tr.go==True].trial_id.values
some_go_trial = go_trials[10]

trial_data = tr[tr.trial_id==some_go_trial]
trial_data.head()

In [ ]:
# plot the mean response across cells for this trial, with cells on x-axis and mean dF/F on y-axis
plt.plot(np.sort(trial_data.mean_response.values), 'o')
plt.ylabel('mean dF/F');
plt.xlabel('sorted cells');

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b>Task 6.4: Explore the flash response dataframe</b>

<p>What is in the <code>flash_response_df</code> attribute of the session object? What are the columns? What are the rows?  How is it different from the <code>stimulus_presentations</code> table?

<p>The <code>flash_response_df</code> contains the cell responses for individual stimulus presentations, aka flashes. It contains the <code>mean_response</code> of every cell in a 500ms window after every stimulus onset, for all stimulus presentations during the behavior session.  

</div>

In [ ]:
# get the flash response dataframe and assign it to 'fr'


In [ ]:
# whats in the flash response dataframe?


<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b>Task 6.5: Plot the mean trace for each image across all flashes</b>

<p>1) Select one image and plot the mean response across all cells using the <code>dff_trace</code> column. Put the <code>image_name</code> in the title. 
    
<p>2) Plot the mean response to all images on the same figure. 

</div>

In [ ]:
# plot the mean trace across all cells for one image
image_names = fr.image_name.unique()
image_name = image_names[0]
traces = fr[fr.image_name==image_name].dff_trace
mean_image_trace = np.mean(traces, axis=0)
time_seconds = fr.iloc[0].dff_trace_timestamps  - fr.iloc[0].start_time
plt.plot(time_seconds, mean_image_trace);
plt.title(image_name);
plt.ylabel('dF/F')
plt.xlabel('time relative to stimulus onset (sec)');

In [ ]:
# plot the mean trace across cells separately for each image
image_names = np.sort(fr.image_name.unique())

fig, ax = plt.subplots()
for image_name in image_names: 
    traces = fr[fr.image_name==image_name].dff_trace
    mean_image_trace = np.mean(traces, axis=0)
    time_seconds = fr.iloc[0].dff_trace_timestamps  - fr.iloc[0].start_time
    ax.plot(time_seconds, mean_image_trace, label=image_name);
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel('dF/F')
ax.set_xlabel('time relative to stimulus onset (sec)');

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">
    <p><b>Task 6.6: Plot one cell's response to each image across all flashes</b>
    
<p>Create the same plot for a single cell rather than the whole population. 
    
<p>What does it look like for different cells?
    
</div>

In [ ]:
# plot the mean trace across images for one cell
cell_specimen_ids = fr.cell_specimen_id.unique()
cell_specimen_id = cell_specimen_ids[4]
cell_data = fr[fr.cell_specimen_id==cell_specimen_id]

fig, ax = plt.subplots()
image_names = np.sort(fr.image_name.unique())
for image_name in image_names: # loop through images
    traces = cell_data[cell_data.image_name==image_name].dff_trace # get traces for one image
    mean_image_trace = np.mean(traces, axis=0)
    time_seconds = fr.iloc[0].dff_trace_timestamps  - fr.iloc[0].start_time
    ax.plot(time_seconds, mean_image_trace, label=image_name);
ax.legend(bbox_to_anchor=(1,1))
ax.set_ylabel('dF/F');
ax.set_xlabel('time relative to stimulus onset (sec)');